In [2]:
import sympy
import numpy as np
from sympy import Matrix, eye, symbols, degree, Poly, fps, Function, simplify, rsolve, init_printing, solve 
from sympy import expand, Abs, limit, sympify
from collections import Counter

## Finding Taylor Series Coefficients 

The generating function we obtain is a rational function, hence it can be represented as 



## Solving Recurrences

The relation we obtain in the 
pathComplexity algorithm is a linear 
homogeneous recurrence relation, 

$$ a_n = c_1a_{n-1} + \cdots + c_ka_{n-k}. $$

To solve this, we'll look for solutions of form $a_n = r^n.$
Suppose 
$$ r^n = c_1r^{n-1} + \cdots + c_kr^{n -k}. $$
Rearranging yields the characteristic equation,
$$ r^k - c_1r^{k-1} - \cdots - c_k = 0. $$ 

We know $r$ is a solution to the characteristic
equation if and only $r^n$ is a solution to 
recurrence. 

Further, any linear combinations of solutions 
to characteristic equation are of solutions to the characteristic equation. Thus factoring the characteristic equation into 

$$ (r - r_1)^{m_1} \cdots (r - r_k)^{m_k} = 0 $$

yields solution

$$ a_n = \sum_{j=1}^k \sum_{i = 0}^{m_j} n^i r_j^n. $$

In [34]:
# Define symbolic terms 
n = symbols('n')
f = Function('f')

# Make a recurrence (equivalent to str(<simpify-expr>))
recurrence = "-1*f(n-1) + 1.0*f(n - 1) + 7*f(n - 2)"

# Get the coefficients 
coeffs = [float(term.strip().split("*")[0]) for term in str(recurrence).split("+")]

# Normalize such that leading coefficient is 1
leadingCoeff = coeffs[0]
coeffs = list(map(lambda coeff: coeff / leadingCoeff, coeffs))

# Find the roots of the characteristic equation 
characteristicEqtnCoeffs = [1.] + [-val for val in coeffs[1:]]
roots = np.roots(characteristicEqtnCoeffs)

# Compute the multiplicy of each root
rootsWithMultiplicites = Counter(roots)

# Compute the coefficients of a_n as a list

a_n = []
for root in rootsWithMultiplicites.keys():
    for i in range(0, rootsWithMultiplicites[root]):
        a_n += [f"(n**{i})*{root}**n"]
        
print(a_n)


['(n**0)*(-0.49999999999999994+2.5980762113533156j)**n', '(n**0)*(-0.49999999999999994-2.5980762113533156j)**n']


# Path Complixity Using Sympy

In [138]:
L = [[0, 0, 1], [0, 0, 0], [0, 1, 0]]
L[1][1] = 1
A = Matrix(L)
print(A)

Matrix([[0, 0, 1], [0, 1, 0], [0, 1, 0]])


In [142]:
t = symbols('t')
dimension = 3
X = eye(dimension) - A*t
print(X)

Matrix([[1, 0, -t], [0, -t + 1, 0], [0, -t, 1]])


In [238]:
X_sub = X.copy()
X_sub.col_del(0)
X_sub.row_del(1)

In [239]:
generatingFunction = X_sub.det() / ((-1)**(1+2) * X.det())
print(generatingFunction)

-t**2/(t - 1)


In [240]:
denominator = Poly(((-1)**(1+2) * X.det()))
print(denominator)

Poly(t - 1, t, domain='ZZ')


In [150]:
recurrenceDegree = degree(denominator, gen=t) + 1 
print(recurrenceDegree)

2


In [153]:
recurrenceKernel = denominator.all_coeffs()[::-1]
print(recurrenceKernel)

[-1, 1]


In [241]:
# We want the first 2*dimension + 1 many coefficients.
def fact(n):
    if n == 0: 
        return 1
    return n*fact(n-1)

f = generatingFunction
taylorCoeffs = []
n=0
for i in range(0, 2*dimension + 1):
    taylorCoeffs.append(f.replace('t', 0) / fact(n))
    f = f.diff()
    n += 1 
    
print(taylorCoeffs)



[0, 0, 1, 1, 1, 1, 1]


In [151]:
baseCases = taylorCoeffs[dimension :
                         dimension + recurrenceDegree - 1]
print(baseCases)

[1]


In [154]:
# Should have as many things as the recurrenceKernel
lRange = Matrix(list(range(0, recurrenceDegree)))
n = symbols('n')
nRange = Matrix([n for _ in range(0, recurrenceDegree)])
f = Function('f')
A = Matrix(list(map(f, nRange - lRange))).dot(Matrix(recurrenceKernel))
print(A)

-f(n) + f(n - 1)


In [158]:
init_printing()
symbolicSol = rsolve(A, f(n))
print(symbolicSol)

C0


In [168]:
r = simplify(symbolicSol)
print(r.evalf())

C0


In [196]:
# Make a list where each is one of [C0, ... CN] terms
numEquations = "5"
coefficients = symbols("C0:" + numEquations)
terms = [1] # TODO 

In [197]:
factors = [i / j for i, j in zip(terms, coefficients)]
print(factors)

[1/C0]


In [198]:
M = [[fact.replace(n, nval) for fact in factors] for nval in range(1, len(factors)+1)]
M = Matrix(M)
print(M)

Matrix([[1/C0]])


In [210]:
invM = M**-1
print(invM)

Matrix([[C0]])


In [212]:
boundingSolutionTerms = (invM * Matrix(baseCases)).dot(Matrix(factors))
print(boundingSolutionTerms)

1


In [225]:
boundingSolutionTerms = expand(boundingSolutionTerms)
print(boundingSolutionTerms)
s = str(boundingSolutionTerms)

n**2
n**2


In [237]:
# Replace all complex numbers with their absolute values

# Replace all instances of x^n with abs(x)^n

# Split terms on '+'
# TEST: expr = n**2 + n + 1
s = str(expr)
terms = [x.strip() for x in s.split("+")]

In [236]:
def bigO(terms, sym):
    
    
    if len(terms) == 1:
        return terms[0]
    
    termOne = terms[0]
    termTwo = terms[1]
    lim = limit(Abs(sympify(termTwo) / sympify(termOne)), n, float('inf'))
    
    if lim == 0:
        return termOne
    
    return bigO(L[1:], sym)
                
bigO(terms, 'n')        

'n**2'